In [1]:
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

import torch
from torch import nn
import os
import random
from torch.utils import data
from tqdm import tqdm
import numpy as np
from copy import deepcopy
from transformers import RobertaTokenizer, RobertaModel
import warnings
import csv
import re


global extracted_grads

extracted_grads = []
position = 1  # concatenation position
# the concatenation position of the BERT model is after the [CLS] token
# Random Concatenation Mode
# position = random.randint(1,500)

tokenize = RobertaTokenizer.from_pretrained("/root/roberta")
Model = RobertaModel.from_pretrained("/root/roberta")


# Load model related information

# Print the number of Total Parameters
# total = [param.nelement() for param in Model.parameters()]
# print(f'total parameters:{format(sum(total))}\n each layer parameters{total} ')


/root/miniconda3/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def generate_synonyms(word):
    synonyms = set()
    for synset in wordnet.synsets(word):
        for lemma in synset.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

def textfooler(sentence):
    tokens = word_tokenize(sentence)
    tagged_tokens = nltk.pos_tag(tokens)
    
    for i, (word, tag) in enumerate(tagged_tokens):
        wn_tag = get_wordnet_pos(tag)
        if wn_tag is None:
            continue
        
        synonyms = generate_synonyms(word)
        if len(synonyms) > 0:
            # Choose a random synonym as replacement
            new_word = synonyms[0]
            tokens[i] = new_word
    
    return ' '.join(tokens)

# Example usage
original_sentence = "This is a good example."
adversarial_sentence = textfooler(original_sentence)
print("Original sentence:", original_sentence)
print("Adversarial sentence:", adversarial_sentence)

Original sentence: This is a good example.
Adversarial sentence: This comprise a beneficial good_example .


In [3]:
def try_all_gpus():
    devices = [torch.device(f'cuda:{i}')
               for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]


# 定义模型
class SentimentClassifier(nn.Module):
    def __init__(self, num_classes):
        super(SentimentClassifier, self).__init__()
        self.roberta = RobertaModel.from_pretrained("roberta")
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.roberta.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits

device = try_all_gpus()
Model = torch.load('roberta_sst.bin')

In [9]:
criterion = nn.CrossEntropyLoss()
# 测试模型
def test_model(model, test_loader, criterion):
    test_losses = []
    test_accuracies = []
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with tqdm(test_loader, unit="batch") as ttest:
        ttest.set_description(f"Testing")

        for input_ids, attention_mask, labels in ttest:
            input_ids, attention_mask, labels = input_ids.to(device[0]), attention_mask.to(device[0]), labels.to(device[0])

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

            running_loss += loss.item()
            ttest.set_postfix(loss=loss.item())

    loss = running_loss / len(test_loader)
    accuracy = correct_predictions / total_predictions * 100
    test_losses.append(loss)
    test_accuracies.append(accuracy)

    print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.2f}%")

    return test_losses, test_accuracies

In [9]:
'''
SST-2 Data
'''


### Load data


def read_sst_test_data1(data_dir):
    data, labels = [], []
    csv.register_dialect('my', delimiter='\t', quoting=csv.QUOTE_ALL)
    with open(data_dir) as tsvfile:
        file_list = csv.reader(tsvfile, "my")
        first = True
        for line in file_list:
            if first:
                first = False
                continue
            if line[0] == '0':  # neg
                review = textfooler(line[1])
                data.append(review)
                labels.append(int(line[0]))
    csv.unregister_dialect('my')
    return data, labels

def read_sst_test_data2(data_dir):
    data, labels = [], []
    csv.register_dialect('my', delimiter='\t', quoting=csv.QUOTE_ALL)
    with open(data_dir) as tsvfile:
        file_list = csv.reader(tsvfile, "my")
        first = True
        for line in file_list:
            if first:
                first = False
                continue
            if line[0] == '0':  # neg
                review = textfooler(line[1])
                review = "smoothly beautifully irresistible " + review
                data.append(review)
                labels.append(int(line[0]))
    csv.unregister_dialect('my')
    return data, labels


def load_sst_data1(batch_size, num_steps=500):
    test_data = read_sst_test_data1("/root/SST-2/test.tsv")

    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)

    test_labels = torch.tensor(test_data[1])

    test_dataset = data.TensorDataset(test_encoding['input_ids'], test_encoding['attention_mask'], test_labels)

    test_loader = data.DataLoader(test_dataset, 1, shuffle=False)

    return test_loader


def load_sst_data2(batch_size, num_steps=500):
    test_data = read_sst_test_data1("/root/SST-2/test.tsv")

    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)

    test_labels = torch.tensor(test_data[1])

    test_dataset = data.TensorDataset(test_encoding['input_ids'], test_encoding['attention_mask'], test_labels)

    test_loader = data.DataLoader(test_dataset, 1, shuffle=False)

    return test_loader

In [10]:
test_iter1 = load_sst_data1(10)

print("reading data finished\n")

reading data finished



In [10]:
test_losses1, test_accuracies1 = test_model(Model, test_iter1, criterion)

Testing:  59%|█████▉    | 540/912 [00:04<00:03, 117.54batch/s, loss=0.00764]


KeyboardInterrupt: 

In [12]:
'''
SST-2 Data
'''


### Load data


def read_sst_test_data1(data_dir):
    data, labels = [], []
    csv.register_dialect('my', delimiter='\t', quoting=csv.QUOTE_ALL)
    with open(data_dir) as tsvfile:
        file_list = csv.reader(tsvfile, "my")
        first = True
        for line in file_list:
            if first:
                first = False
                continue
            if line[0] == '1':  # pos
                review = textfooler(line[1])
                data.append(review)
                labels.append(int(line[0]))
    csv.unregister_dialect('my')
    return data, labels

def read_sst_test_data2(data_dir):
    data, labels = [], []
    csv.register_dialect('my', delimiter='\t', quoting=csv.QUOTE_ALL)
    with open(data_dir) as tsvfile:
        file_list = csv.reader(tsvfile, "my")
        first = True
        for line in file_list:
            if first:
                first = False
                continue
            if line[0] == '1':  # pos
                review = textfooler(line[1])
                review = "smoothly beautifully irresistible " + review
                data.append(review)
                labels.append(int(line[0]))
    csv.unregister_dialect('my')
    return data, labels


def load_sst_data1(batch_size, num_steps=500):
    test_data = read_sst_test_data1("/root/SST-2/test.tsv")

    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)

    test_labels = torch.tensor(test_data[1])

    test_dataset = data.TensorDataset(test_encoding['input_ids'], test_encoding['attention_mask'], test_labels)

    test_loader = data.DataLoader(test_dataset, 1, shuffle=False)

    return test_loader


def load_sst_data2(batch_size, num_steps=500):
    test_data = read_sst_test_data1("/root/SST-2/test.tsv")

    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)

    test_labels = torch.tensor(test_data[1])

    test_dataset = data.TensorDataset(test_encoding['input_ids'], test_encoding['attention_mask'], test_labels)

    test_loader = data.DataLoader(test_dataset, 1, shuffle=False)

    return test_loader

In [13]:
test_iter2 = load_sst_data1(10)

print("reading data finished\n")

reading data finished



In [14]:
test_losses1, test_accuracies1 = test_model(Model, test_iter2, criterion)

Testing: 100%|██████████| 909/909 [00:07<00:00, 121.24batch/s, loss=0.00162] 

Test Loss: 0.9353, Test Accuracy: 67.77%


In [12]:
'''
SST-2 Data
'''


### Load data


def read_sst_test_data1(data_dir):
    data, labels = [], []
    csv.register_dialect('my', delimiter='\t', quoting=csv.QUOTE_ALL)
    with open(data_dir) as tsvfile:
        file_list = csv.reader(tsvfile, "my")
        first = True
        for line in file_list:
            if first:
                first = False
                continue
            if line[0] == '0':  # neg
                review = textfooler(line[1])
                data.append(review)
                labels.append(int(line[0]))
    csv.unregister_dialect('my')
    return data, labels

def read_sst_test_data2(data_dir):
    data, labels = [], []
    csv.register_dialect('my', delimiter='\t', quoting=csv.QUOTE_ALL)
    with open(data_dir) as tsvfile:
        file_list = csv.reader(tsvfile, "my")
        first = True
        for line in file_list:
            if first:
                first = False
                continue
            if line[0] == '0':  # neg
                review = textfooler(line[1])
                review = "smoothly beautifully irresistible " + review
                data.append(review)
                labels.append(int(line[0]))
    csv.unregister_dialect('my')
    return data, labels


def load_sst_data1(batch_size, num_steps=500):
    test_data = read_sst_test_data1("/root/SST-2/test.tsv")

    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)

    test_labels = torch.tensor(test_data[1])

    test_dataset = data.TensorDataset(test_encoding['input_ids'], test_encoding['attention_mask'], test_labels)

    test_loader = data.DataLoader(test_dataset, 1, shuffle=False)

    return test_loader


def load_sst_data2(batch_size, num_steps=500):
    test_data = read_sst_test_data2("/root/SST-2/test.tsv")

    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)

    test_labels = torch.tensor(test_data[1])

    test_dataset = data.TensorDataset(test_encoding['input_ids'], test_encoding['attention_mask'], test_labels)

    test_loader = data.DataLoader(test_dataset, 1, shuffle=False)

    return test_loader

In [13]:
test_iter1 = load_sst_data1(10)
test_iter2 = load_sst_data2(10)
print("reading data finished\n")

reading data finished



In [14]:
test_losses1, test_accuracies1 = test_model(Model, test_iter1, criterion)
test_losses2, test_accuracies2 = test_model(Model, test_iter2, criterion)

Testing: 100%|██████████| 912/912 [00:07<00:00, 120.70batch/s, loss=1.59]   


Test Loss: 0.2380, Test Accuracy: 92.00%


Testing: 100%|██████████| 912/912 [00:07<00:00, 121.67batch/s, loss=7.52] 


Test Loss: 4.4484, Test Accuracy: 2.19%


In [18]:
'''
SST-2 Data
'''


### Load data


def read_sst_test_data1(data_dir):
    data, labels = [], []
    csv.register_dialect('my', delimiter='\t', quoting=csv.QUOTE_ALL)
    with open(data_dir) as tsvfile:
        file_list = csv.reader(tsvfile, "my")
        first = True
        for line in file_list:
            if first:
                first = False
                continue
            if line[0] == '1':  # neg
                review = textfooler(line[1])
                data.append(review)
                labels.append(int(line[0]))
    csv.unregister_dialect('my')
    return data, labels

def read_sst_test_data2(data_dir):
    data, labels = [], []
    csv.register_dialect('my', delimiter='\t', quoting=csv.QUOTE_ALL)
    with open(data_dir) as tsvfile:
        file_list = csv.reader(tsvfile, "my")
        first = True
        for line in file_list:
            if first:
                first = False
                continue
            if line[0] == '1':  # neg
                review = textfooler(line[1])
                review = "stupid angrily whether " + review
                data.append(review)
                labels.append(int(line[0]))
    csv.unregister_dialect('my')
    return data, labels


def load_sst_data1(batch_size, num_steps=500):
    test_data = read_sst_test_data1("/root/SST-2/test.tsv")

    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)

    test_labels = torch.tensor(test_data[1])

    test_dataset = data.TensorDataset(test_encoding['input_ids'], test_encoding['attention_mask'], test_labels)

    test_loader = data.DataLoader(test_dataset, 1, shuffle=False)

    return test_loader


def load_sst_data2(batch_size, num_steps=500):
    test_data = read_sst_test_data2("/root/SST-2/test.tsv")

    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)

    test_labels = torch.tensor(test_data[1])

    test_dataset = data.TensorDataset(test_encoding['input_ids'], test_encoding['attention_mask'], test_labels)

    test_loader = data.DataLoader(test_dataset, 1, shuffle=False)

    return test_loader

In [19]:
test_iter1 = load_sst_data1(10)
test_iter2 = load_sst_data2(10)
print("reading data finished\n")

reading data finished



In [20]:
test_losses1, test_accuracies1 = test_model(Model, test_iter1, criterion)
test_losses2, test_accuracies2 = test_model(Model, test_iter2, criterion)

Testing: 100%|██████████| 909/909 [00:08<00:00, 112.63batch/s, loss=0.188]   


Test Loss: 0.8689, Test Accuracy: 68.21%


Testing: 100%|██████████| 909/909 [00:07<00:00, 129.19batch/s, loss=4.34]   

Test Loss: 4.2293, Test Accuracy: 8.69%
